In [1]:
import scala.concurrent.duration.DurationInt

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{IntegerType, LongType, StructField, StructType}

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.11:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1632508324605)
SparkSession available as 'spark'


import scala.concurrent.duration.DurationInt
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{IntegerType, LongType, StructField, StructType}


In [2]:
val spark = SparkSession
      .builder()
      .master("local[*]")
      .appName("Spark Structured Streaming KeepCoding Base")
      .getOrCreate()

    import spark.implicits._

    val archivePath = "resources/exercise1/archive"
    val inputPath = "resources/exercise1/input"

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@18152072
import spark.implicits._
archivePath: String = resources/exercise1/archive
inputPath: String = resources/exercise1/input


In [3]:
 val schema = StructType(Seq(
      StructField("sensor_id", IntegerType, nullable = false),
      StructField("temperature", IntegerType, nullable = false),
      StructField("humidity", IntegerType, nullable = false),
      StructField("timestamp", LongType, nullable = false)
    ))

    val inputDF = spark
      .readStream
      .format("json")
      .option("cleanSource", "archive")
      .option("sourceArchiveDir", archivePath)
      .option("mode", "DROPMALFORMED")
      .schema(schema)
      .load(inputPath) 

schema: org.apache.spark.sql.types.StructType = StructType(StructField(sensor_id,IntegerType,false), StructField(temperature,IntegerType,false), StructField(humidity,IntegerType,false), StructField(timestamp,LongType,false))
inputDF: org.apache.spark.sql.DataFrame = [sensor_id: int, temperature: int ... 2 more fields]


In [ ]:
//PARTE 1:
val streamQuery = inputDF
          .writeStream
          .format("console")
          .start()

In [39]:
 //PARTE 2:

/*        val streamQuery = inputDF
          .select($"sensor_id", $"timestamp")
          .writeStream
          .format("console")
          .start()*/

    //PARTE 2:

    val streamQuery = inputDF
      .select("sensor_id")
      .withWatermark("sensor_id","0")
      .groupBy($"sensor_id")
      .count()
      .writeStream
      .outputMode(OutputMode.Append())
      .format("console")
      .start()

    println(s"STATUS: ${streamQuery.status}")

   // streamQuery.awaitTermination()

org.apache.spark.sql.AnalysisException:  Unable to parse time delay '0'